In [59]:
# %cd /content/drive/MyDrive/Colab Notebooks/05_추천시스템/추천시스템_프로젝트
# !unzip -qq "/content/drive/MyDrive/수업자료/00_dataset/BX-CSV.zip"

In [60]:
# !pip install surprise
import surprise 

print(surprise.__version__)

1.1.3


In [61]:
from surprise import SVD
from surprise import Dataset 
from surprise import accuracy 
from surprise.model_selection import train_test_split

In [62]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [63]:
rating_df = pd.read_csv("BX-Book-Ratings.csv",sep=';', encoding='latin_1',error_bad_lines=False)
book_df = pd.read_csv("BX-Books.csv",sep=';', encoding='latin_1',error_bad_lines=False)
user_df = pd.read_csv("BX-Users.csv",sep=';', encoding='latin_1',error_bad_lines=False)

/var/folders/mv/9j6rfk0s3hlc_vvyzx5b32fw0000gn/T/ipykernel_77119/2848770525.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  rating_df = pd.read_csv("BX-Book-Ratings.csv",sep=';', encoding='latin_1',error_bad_lines=False)
/var/folders/mv/9j6rfk0s3hlc_vvyzx5b32fw0000gn/T/ipykernel_77119/2848770525.py:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  book_df = pd.read_csv("BX-Books.csv",sep=';', encoding='latin_1',error_bad_lines=False)
b'Skipping line 6452: expected 8 fields, saw 9\nSkipping line 43667: expected 8 fields, saw 10\nSkipping line 51751: expected 8 fields, saw 9\n'
b'Skipping line 92038: expected 8 fields, saw 9\nSkipping line 104319: expected 8 fields, saw 9\nSkipping line 121768: expected 8 fields, saw 9\n'
b'Skipping line 144058: expected 8 fields, saw 9\nSkipping line 150789: expec

In [64]:
print(rating_df.info())
print(book_df.info())
print(user_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1149780 entries, 0 to 1149779
Data columns (total 3 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   User-ID      1149780 non-null  int64 
 1   ISBN         1149780 non-null  object
 2   Book-Rating  1149780 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 26.3+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271360 entries, 0 to 271359
Data columns (total 8 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   ISBN                 271360 non-null  object
 1   Book-Title           271360 non-null  object
 2   Book-Author          271359 non-null  object
 3   Year-Of-Publication  271360 non-null  object
 4   Publisher            271358 non-null  object
 5   Image-URL-S          271360 non-null  object
 6   Image-URL-M          271360 non-null  object
 7   Image-URL-L          271357 non-null  object
dtypes:

In [65]:
rating_df.head(1)

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0


In [66]:
book_df.head(1)

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...


In [67]:
user_df.head(1)

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN


In [68]:
rating_up1 = rating_df[rating_df['Book-Rating']>=1]

In [69]:
original_data_size = len(rating_df)
filtered_data_size = len(rating_up1)

print(f'original_data_size: {original_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / original_data_size:.2%}')

original_data_size: 1149780, filtered_data_size: 433671
Ratio of Remaining Data is 37.72%


In [70]:
book_rating_df = pd.merge(rating_up1, book_df[['ISBN', 'Book-Title']], on = 'ISBN')
book_rating_df['Book-Rating'] = book_rating_df['Book-Rating'].astype(float)
book_rating_df

,User-ID,ISBN,Book-Rating,Book-Title
0,276726,0155061224,5.0,Rites of Passage
1,276729,052165615X,3.0,Help!: Level 1
2,276729,0521795028,6.0,The Amsterdam Connection : Level 4 (Cambridge ...
3,276744,038550120X,7.0,A Painted House
4,11676,038550120X,10.0,A Painted House
...,...,...,...,...
383837,276688,0425156737,2.0,Graven Images
383838,276688,0671563149,6.0,MUDDY WATER (Peter Bartholomew Mysteries)
383839,276688,1575660792,7.0,Gray Matter
383840,276704,0380796155,5.0,White Abacus


- Surprise 이용(SVD)

In [71]:
from surprise import Dataset, Reader

reader = Reader(rating_scale=(0, 10))
data = Dataset.load_from_df(rating_up1[['User-ID', 'ISBN', 'Book-Rating']], reader=reader)
trainset, testset = train_test_split(data, test_size=.25, random_state=0) 

algo = SVD(n_factors=1000, random_state=0)
algo.fit(trainset) 
predictions = algo.test( testset )
accuracy.rmse(predictions)

RMSE: 1.6859


1.6858824573073767

- 교차 검증

In [72]:
from surprise.model_selection import cross_validate
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.6831  1.6777  1.6816  1.6826  1.6781  1.6806  0.0023  
MAE (testset)     1.3104  1.3079  1.3111  1.3125  1.3102  1.3104  0.0015  
Fit time          19.62   19.57   18.60   18.38   18.62   18.96   0.52    
Test time         0.38    0.40    0.38    0.38    0.36    0.38    0.01    


{'test_rmse': array([1.68311301, 1.67769253, 1.68158392, 1.68262004, 1.6781471 ]),
 'test_mae': array([1.31035253, 1.30787223, 1.3111496 , 1.31250578, 1.31023622]),
 'fit_time': (19.618272066116333,
  19.567262172698975,
  18.603357791900635,
  18.384286880493164,
  18.620519876480103),
 'test_time': (0.3849799633026123,
  0.4028921127319336,
  0.3786149024963379,
  0.38422107696533203,
  0.3634459972381592)}

- GridSearchCV로 파라미터 탐색

In [73]:
from surprise.model_selection import GridSearchCV

# 최적화할 파라미터를 딕셔너리 형태로 지정.
param_grid = {'n_epochs': [40, 60], 'n_factors': [150, 250, 500] }

# CV를 3개 폴드 세트로 지정, 성능 평가는 rmse, mse로 수행하도록 GridSearchCV 구성
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=2)
gs.fit(data)

# 최고 RMSE Evaluation 점수와 그때의 하이퍼 파라미터
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

1.6596191061401997
{'n_epochs': 40, 'n_factors': 150}


- 개인화 영화 추천 시스템 구축

In [74]:
reader = Reader(rating_scale=(0, 10))
data = Dataset.load_from_df(rating_up1[['User-ID', 'ISBN', 'Book-Rating']], reader=reader)

#전체 데이터를 학습데이터로 생성함.
trainset = data.build_full_trainset()

In [75]:
algo = SVD(n_epochs=40, n_factors=150, random_state=0)
algo.fit(trainset)

In [76]:
def get_unseen_surprise(ratings, movies, userId):
    #입력값으로 들어온 User-ID에 해당하는 사용자가 평점을 매긴 모든 영화를 리스트로 생성
    seen_movies = ratings[ratings['User-ID']== userId]['ISBN'].tolist()
    
    # 모든 영화들의 movieId를 리스트로 생성. 
    total_movies = movies['ISBN'].tolist()
    
    # 모든 영화들의 movieId중 이미 평점을 매긴 영화의 movieId를 제외하여 리스트로 생성
    unseen_movies= [movie for movie in total_movies if movie not in seen_movies]
    print('평점 매긴 권수:',len(seen_movies), '추천대상 권수:',len(unseen_movies), \
          '전체 권수:',len(total_movies))
    
    return unseen_movies

unseen_movies = get_unseen_surprise(rating_up1, book_df, 264321)

평점 매긴 권수: 285 추천대상 권수: 271087 전체 권수: 271360


In [77]:
def recomm_book_by_surprise(algo, userId, unseen_movies, top_n=10):
    # 알고리즘 객체의 predict() 메서드를 평점이 없는 영화에 반복 수행한 후 결과를 list 객체로 저장
    predictions = [algo.predict(int(userId), str(bookId)) for bookId in unseen_movies]
    
    # predictions list 객체는 surprise의 Predictions 객체를 원소로 가지고 있음.
    # [Prediction(uid='264321', iid='1', est=3.6264321), Prediction(uid='264321', iid='2', est=2.2643218),,,,]
    # 이를 est 값으로 정렬하기 위해서 아래의 sortkey_est 함수를 정의함.
    # sortkey_est 함수는 list 객체의 sort() 함수의 키 값으로 사용되어 정렬 수행.
    def sortkey_est(pred):
        return pred.est
    
    # sortkey_est( ) 반환값의 내림 차순으로 정렬 수행하고 top_n개의 최상위 값 추출.
    predictions.sort(key=sortkey_est, reverse=True)
    top_predictions= predictions[:top_n]
    
    # top_n으로 추출된 영화의 정보 추출. 영화 아이디, 추천 예상 평점, 제목 추출
    top_book_ids = [ str(pred.iid) for pred in top_predictions]
    top_book_rating = [ pred.est for pred in top_predictions]
    top_book_titles = book_df[book_df['ISBN'].isin(top_book_ids)]['Book-Title']
    top_book_preds = [ (id, title, rating) for id, title, rating in zip(top_book_ids, top_book_titles, top_book_rating)]
    
    return top_book_preds

In [87]:
book_rating_df[book_rating_df['User-ID']==88705]

,User-ID,ISBN,Book-Rating,Book-Title
280821,88705,0525144609,2.0,Leviathan


In [88]:
book_df[book_df['Book-Title']=='Leviathan']

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
119139,0525144609,Leviathan,John Gordon Davis,1976,Dutton,http://images.amazon.com/images/P/0525144609.0...,http://images.amazon.com/images/P/0525144609.0...,http://images.amazon.com/images/P/0525144609.0...


In [85]:
test_ids = [88705, 264321, 182459, 161936, 226482, 1903, 3346]
for i in test_ids:
    print('id: {}'.format(i))
    unseen_movies = get_unseen_surprise(rating_up1, book_df, i)
    top_book_preds = recomm_book_by_surprise(algo, i, unseen_movies, top_n=5)
    print('##### Top-10 추천 영화 리스트 #####')

    for top_movie in top_book_preds:
        print(top_movie[0]," - ",top_movie[1], ":", top_movie[2])
    print('-'*20)

id: 88705
평점 매긴 권수: 1 추천대상 권수: 271359 전체 권수: 271360
##### Top-10 추천 영화 리스트 #####
0439425220  -  Anne of Green Gables (Anne of Green Gables Novels (Paperback)) : 8.784260020002101
055321313X  -  Where the Sidewalk Ends : Poems and Drawings : 8.641982435815853
3499224615  -  Lonesome Dove : 8.581625969158333
067168390X  -  Maus 1. Mein Vater kotzt Geschichte aus. Die Geschichte eines Ã?Â?berlebenden. : 8.558448465550228
0060256672  -  Harry Potter and the Chamber of Secrets Postcard Book : 8.501018475820446
--------------------
id: 264321
평점 매긴 권수: 285 추천대상 권수: 271087 전체 권수: 271360
##### Top-10 추천 영화 리스트 #####
0316284955  -  The Hitchhiker's Guide to the Galaxy : 8.424372429053479
3404148665  -  Stupid White Men ...and Other Sorry Excuses for the State of the Nation! : 8.120634796861278
0060392452  -  White Oleander : A Novel (Oprah's Book Club) : 8.051389381618646
0440414806  -  Holes (Yearling Newbery) : 7.855908869583674
0345391802  -  Illuminati. : 7.758313804364899
-----------------